In [ ]:
import cv2
import numpy as np
import os

from imgaug import augmenters as iaa
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage


# Define the data augmentation pipeline
augmenter = iaa.Sequential([
    iaa.Fliplr(0.5),  # horizontally flip 50% of the images
    iaa.Affine(rotate=(-10, 10)),  # rotate images by -10 to +10 degrees
    iaa.GaussianBlur(sigma=(0, 1.0)),  # add random gaussian blur
    iaa.AdditiveGaussianNoise(scale=(0, 0.05 * 255)),  # add gaussian noise
])


# Define the path to the image folder
img_folder_path = "path/to/image/folder"

# Define the path to the annotation file
annotation_file_path = "path/to/annotation/file"

# Load the annotations
with open(annotation_file_path, "r") as f:
    annotations = f.readlines()

# Loop over the images and their annotations
for annotation in annotations:
    # Split the annotation into its components
    img_path, *bboxes = annotation.strip().split(" ")
    bboxes = np.array([list(map(float, bbox.split(","))) for bbox in bboxes])

    # Load the image
    img = cv2.imread(os.path.join(img_folder_path, img_path))

    # Convert the bounding boxes to imgaug format
    bounding_boxes = [BoundingBox(x1=bbox[0], y1=bbox[1], x2=bbox[2], y2=bbox[3], label="tree") for bbox in bboxes]
    bbs = BoundingBoxesOnImage(bounding_boxes, shape=img.shape)

    # Apply the data augmentation
    img_aug, bbs_aug = augmenter(image=img, bounding_boxes=bbs)

    # Convert the augmented bounding boxes back to YOLO format
    bboxes_aug = np.array([[bbox.x1, bbox.y1, bbox.x2, bbox.y2] for bbox in bbs_aug])

    # Save the augmented image and its annotations
    img_path_aug = os.path.splitext(img_path)[0] + "_augmented.jpg"
    cv2.imwrite(os.path.join(img_folder_path, img_path_aug), img_aug)

    with open(os.path.join(img_folder_path, os.path.splitext(img_path)[0] + "_augmented.txt"), "w") as f:
        for bbox_aug in bboxes_aug:
            f.write("0 {:.6f} {:.6f} {:.6f} {:.6f}\n".format(
                (bbox_aug[0] + bbox_aug[2]) / 2 / img_aug.shape[1],
                (bbox_aug[1] + bbox_aug[3]) / 2 / img_aug.shape[0],
                (bbox_aug[2] - bbox_aug[0]) / img_aug.shape[1],
                (bbox_aug[3] - bbox_aug[1]) / img_aug.shape[0]
            ))
